In [0]:
%python
import re

def to_snake_case(name: str) -> str:
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    name = re.sub('([a-z0-9])([A-Z])', r'\1_\2', name)
    return name.lower()

In [0]:
%python
base_path = "/Volumes/adventureworks/dataset/raw/sales/"
bronze_path = "adventureworks.bronze"

tables = [
    f.name.removesuffix(".csv")
    for f in dbutils.fs.ls(base_path)
        if f.name.endswith(".csv")
]

In [0]:
%python
for table in tables:
    table_name = to_snake_case(table)
    
    df = (spark
            .read
            .format("csv")
            .option("header", "true")
            .option("sep", ";")
            .option("encoding", "ISO-8859-1")
            .load(f'{base_path}/{table}.csv')
        )
    
    df.write.mode('overwrite').saveAsTable(f'{bronze_path}.{table_name}')
    
    df = spark.read.table(f"{bronze_path}.{table_name}")
    df = df.toDF(*[to_snake_case(c) for c in df.columns])
    df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(f'{bronze_path}.{table_name}')